In [1]:
# Build Graph
# graphframe
sc.addPyFile('/usr/local/Cellar/apache-spark/2.4.1/libexec/jars/graphframes-0.7.0-spark2.4-s_2.11.jar')

# modules
import sys
from graphframes import *
import pyspark.sql.functions as f
from pyspark.sql.functions import lit

# library 
sys.path.append("/Users/jeanxu/Documents/UniLU/0_MasterThesis/5_Notes/QA_Notes/1_Test/python/project/library")
import get_answer_types_from_questions as gtQType
import get_all_subjects_predicates_from_questions as gtQSub
from stanfordcorenlp import StanfordCoreNLP

In [2]:
# config
verbose = 0
nlp = StanfordCoreNLP(r'/Users/jeanxu/PycharmProjects/QUEST_TerminalTest/Code/stanford-corenlp-full-2018-10-05')
prune = 5
question = 'when is national day of England'

In [3]:
# Extracting potential answer type from question
f1 = 'question_type.txt'
gtQType.get_answer_type_main(question,f1,nlp)

In [4]:
# Extracting query terms from question.
s1 = 'question_subject_predicate.txt'
gtQSub.get_sub_pred_ques_main(question,s1,nlp)
aux_list = set()
for l in open('auxiliary_verb_list.txt'):
    l = l.strip()
    aux_list.add(l.lower())

In [5]:
# Read entities of question
s11 = open(s1, 'r')
q_ent = set()
type_qent = {}
for line in s11:
    line = line.strip()
    line = line.split()
    s = line[0]
    for i in range(1, len(line) - 1):
        s += ' ' + line[i]
    q_ent.add(s.lower())
    type_qent[s.lower()] = line[len(line) - 1]
if verbose:
    print "Query terms ->", len(q_ent), q_ent
if len(q_ent) > prune:
    if verbose:
        print "Pruning.."
    p = frozenset(q_ent)
    for s in p:
        if type_qent[s] == 'P':
            q_ent.remove(s)

In [6]:
q_ent

{'england', 'is', 'national day'}

In [7]:
type_qent

{'england': 'NE', 'is': 'P', 'national day': 'NE'}

In [8]:
p = frozenset(q_ent)
for s in p:
    if s in aux_list and type_qent[s] == 'P':
        q_ent.remove(s)

In [9]:
p

frozenset({'england', 'is', 'national day'})

In [10]:
# file path
File_verticesTextRDD = "/Users/jeanxu/Documents/UniLU/0_MasterThesis/4_Spark/ReferenceFromPaul/Graph-Small/Vertices/part-*"
File_edgesTextRDD = "/Users/jeanxu/Documents/UniLU/0_MasterThesis/4_Spark/ReferenceFromPaul/Graph-Small/Edges/part-*"


In [11]:
######### load vertices
verticesText0 = spark.read.csv(File_verticesTextRDD, header='false', inferSchema='false', sep='\t')

verticesText1 = verticesText0.select("_c1","_c0","_c2","_c3","_c4")\
.withColumnRenamed("_c0", "nodeType").withColumnRenamed("_c1", "id")\
.withColumnRenamed("_c2", "attr1").withColumnRenamed("_c3", "attr2")\
.withColumnRenamed("_c4", "attr3")

In [ ]:
##Sample test

In [105]:
fraction = 0.1
# data_sample = data_from_file_conv.sample(False, fraction, 666)
smaple_test = verticesText1.sample(False, fraction, 666)

In [106]:
smaple_test.count()

858943

In [108]:
verticesText1.count()

8593092

In [107]:
smaple_test.show()

+---+--------+-----+--------------------+-------------------+
| id|nodeType|attr1|               attr2|              attr3|
+---+--------+-----+--------------------+-------------------+
|  0|Sentence|    1|Politics of Burun...|Politics of Burundi|
|  8|Sentence|    9|He declared himse...|Politics of Burundi|
| 20|Sentence|   21|The Senate ("Séna...|Politics of Burundi|
| 37|Sentence|   12|Explicitly, "beha...| Abstract data type|
| 40|Sentence|   15|An "interface" ty...| Abstract data type|
| 42|Sentence|   17|An abstract queue...| Abstract data type|
| 43|Sentence|   18|There would be no...| Abstract data type|
| 54|Sentence|   29|In the philosophy...| Abstract data type|
| 55|Sentence|   30|Some operations m...| Abstract data type|
| 61|Sentence|   36|In this definitio...| Abstract data type|
| 66|Sentence|   41|Note that this de...| Abstract data type|
|102|Sentence|   77|In this way, the ...| Abstract data type|
|106|Sentence|   81|This gives a grea...| Abstract data type|
|139|Sen

In [12]:
verticesText1.show()

+---+--------+-----+--------------------+-------------------+
| id|nodeType|attr1|               attr2|              attr3|
+---+--------+-----+--------------------+-------------------+
|  0|Sentence|    1|Politics of Burun...|Politics of Burundi|
|  1|Sentence|    2|Executive power i...|Politics of Burundi|
|  2|Sentence|    3|Legislative power...|Politics of Burundi|
|  3|Sentence|    4|The political lan...|Politics of Burundi|
|  4|Sentence|    5|The current Presi...|Politics of Burundi|
|  5|Sentence|    6|Nkurunziza was th...|Politics of Burundi|
|  6|Sentence|    7|In November 1995,...|Politics of Burundi|
|  7|Sentence|    8|In July 1996, for...|Politics of Burundi|
|  8|Sentence|    9|He declared himse...|Politics of Burundi|
|  9|Sentence|   10|Widespread condem...|Politics of Burundi|
| 10|Sentence|   11|Buyoya agreed in ...|Politics of Burundi|
| 11|Sentence|   12|Nonetheless, figh...|Politics of Burundi|
| 12|Sentence|   13|In June 1998, Buy...|Politics of Burundi|
| 13|Sen

In [12]:
######### load edges
edgesText0 = spark.read.csv(File_edgesTextRDD)
edgesText0 = edgesText0.select(f.translate(f.col("_c0"), "Edge(", "").alias("src"), "_c1", f.translate(f.col("_c2"), ")", "").alias("label"))
edgesText1 = edgesText0.select("*").withColumnRenamed("_c1", "dst")
verticesText1J = verticesText1

In [14]:
verticesText1J.show()

+---+--------+-----+--------------------+-------------------+
| id|nodeType|attr1|               attr2|              attr3|
+---+--------+-----+--------------------+-------------------+
|  0|Sentence|    1|Politics of Burun...|Politics of Burundi|
|  1|Sentence|    2|Executive power i...|Politics of Burundi|
|  2|Sentence|    3|Legislative power...|Politics of Burundi|
|  3|Sentence|    4|The political lan...|Politics of Burundi|
|  4|Sentence|    5|The current Presi...|Politics of Burundi|
|  5|Sentence|    6|Nkurunziza was th...|Politics of Burundi|
|  6|Sentence|    7|In November 1995,...|Politics of Burundi|
|  7|Sentence|    8|In July 1996, for...|Politics of Burundi|
|  8|Sentence|    9|He declared himse...|Politics of Burundi|
|  9|Sentence|   10|Widespread condem...|Politics of Burundi|
| 10|Sentence|   11|Buyoya agreed in ...|Politics of Burundi|
| 11|Sentence|   12|Nonetheless, figh...|Politics of Burundi|
| 12|Sentence|   13|In June 1998, Buy...|Politics of Burundi|
| 13|Sen

In [13]:
edgesText2 = edgesText1.join(verticesText1.select("id","nodeType"), edgesText1.src == verticesText1.select("id","nodeType").id, "inner")
edgesText2 = edgesText2.withColumnRenamed("id", "src_id").withColumnRenamed("nodeType", "src_nodeType")
edgesText3 = edgesText2.join(verticesText1.select("id","nodeType"), edgesText2.dst == verticesText1.select("id","nodeType").id, "inner")
edgesText3 = edgesText3.withColumnRenamed("id", "dst_id").withColumnRenamed("nodeType", "dst_nodeType")

In [16]:
edgesText3.show()

+-------+-------+-------------------+-------+------------+-------+------------+
|    src|    dst|              label| src_id|src_nodeType| dst_id|dst_nodeType|
+-------+-------+-------------------+-------+------------+-------+------------+
|1234113| 100010|contains the clause|1234113|    Sentence| 100010|      Clause|
|3875588|1000240|contains the clause|3875588|    Sentence|1000240|      Clause|
|3650624|1000280|contains the clause|3650624|    Sentence|1000280|      Clause|
|2068926|1000665|contains the clause|2068926|    Sentence|1000665|      Clause|
| 467648|1000795|contains the clause| 467648|    Sentence|1000795|      Clause|
| 233944|1000839|contains the clause| 233944|    Sentence|1000839|      Clause|
|4022932|1000888|contains the clause|4022932|    Sentence|1000888|      Clause|
|1212133| 100140|contains the clause|1212133|    Sentence| 100140|      Clause|
| 452611|1001866|contains the clause| 452611|    Sentence|1001866|      Clause|
|1487892|1002011|contains the clause|148

In [17]:
######### documnet vertice & edges

In [14]:
### documnets---attributes
doc_verticesText = verticesText1.filter("nodeType = 'Document'")
doc_verticesText = doc_verticesText.withColumn("did", doc_verticesText.id).withColumn("dtitle", doc_verticesText.attr1).drop("attr1").drop("attr2").drop("attr3")

In [19]:
doc_verticesText.show()

+----+--------+----+--------------------+
|  id|nodeType| did|              dtitle|
+----+--------+----+--------------------+
|4526|Document|4526|     Africa Alphabet|
|4527|Document|4527|               Brain|
|4528|Document|4528|     Andreas Aagesen|
|4529|Document|4529|Australian Broadc...|
|4530|Document|4530|The Elephant 6 Re...|
|4531|Document|4531|Albert of Branden...|
|4532|Document|4532|Telecommunication...|
|4533|Document|4533|                 DMA|
|4534|Document|4534|         Alford plea|
|4535|Document|4535|        Claude Piron|
|4536|Document|4536|Full disclosure (...|
|4537|Document|4537|    Anthem of Europe|
|4538|Document|4538|             Cytosol|
|4539|Document|4539|            Barbagia|
|4540|Document|4540|             Calcium|
|4541|Document|4541|   Chomsky hierarchy|
|4542|Document|4542| Critical philosophy|
|4543|Document|4543|   Cultural movement|
|4544|Document|4544|          Braveheart|
|4545|Document|4545|                Bill|
+----+--------+----+--------------

In [15]:
### sentence---attributes
# sen_verticesText = verticesText1.filter("nodeType = 'Sentence'")
sen_edges = edgesText3.filter("label = 'contains the sentence'")
sen_verticesTextJ_test = verticesText1.filter("nodeType = 'Sentence'")
sen_edgesJ = sen_edges.select("src_id","src_nodeType","dst_id","dst_nodeType")

In [21]:
sen_edgesJ.show()

+------+------------+-------+------------+
|src_id|src_nodeType| dst_id|dst_nodeType|
+------+------------+-------+------------+
|626404|    Document|1014942|    Sentence|
|626407|    Document|1015263|    Sentence|
|626411|    Document|1015551|    Sentence|
|626411|    Document|1015552|    Sentence|
|626411|    Document|1015610|    Sentence|
|626413|    Document|1015827|    Sentence|
|626414|    Document|1015968|    Sentence|
|626417|    Document|1016299|    Sentence|
|626417|    Document|1016518|    Sentence|
|626417|    Document|1016816|    Sentence|
|626418|    Document|1017045|    Sentence|
|626418|    Document|1017074|    Sentence|
|626419|    Document|1017462|    Sentence|
|626421|    Document|1017551|    Sentence|
|626422|    Document|1017716|    Sentence|
|626422|    Document|1017805|    Sentence|
|626422|    Document|1017849|    Sentence|
|626422|    Document|1018090|    Sentence|
|626422|    Document|1018100|    Sentence|
|626392|    Document|1018247|    Sentence|
+------+---

In [16]:
# conbine src_id
sen_edgesJ_test1 = sen_edgesJ.join(doc_verticesText, sen_edgesJ.src_id==doc_verticesText.id,"inner")
sen_edgesJ_test1 = sen_edgesJ_test1.drop("id").drop("nodeType")


In [23]:
sen_edgesJ_test1.show()

+-------+------------+-------+------------+-------+--------------+
| src_id|src_nodeType| dst_id|dst_nodeType|    did|        dtitle|
+-------+------------+-------+------------+-------+--------------+
|1019881|    Document|3245237|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245224|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245236|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245232|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245222|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245228|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245238|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245239|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245223|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245229|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245231|    Sentence|1019881|Emperor Ninken|
|1019881|    Document|3245225|    Sentence|1019881|Emperor Nin

In [17]:
# conbine dst_id
sen_verticesTextJ_test1 = sen_edgesJ_test1.join(sen_verticesTextJ_test,sen_edgesJ_test1.dst_id==sen_verticesTextJ_test.id,"inner")
sen_verticesTextJ_test2 = sen_verticesTextJ_test1.drop("attr2").drop("attr3")
sen_verticesText = sen_verticesTextJ_test2.drop("src_id").drop("src_nodeType").drop("dst_id").drop("dst_nodeType")
sen_verticesText = sen_verticesText.withColumnRenamed("attr1","sid")
sen_verticesText = sen_verticesText.select("id","nodeType","did","dtitle","sid")

In [25]:
sen_verticesText.show()

+-------+--------+------+--------------------+---+
|     id|nodeType|   did|              dtitle|sid|
+-------+--------+------+--------------------+---+
|1014942|Sentence|626404|              Genome|102|
|1015263|Sentence|626407|             Alcohol| 99|
|1015551|Sentence|626411|             Cartoon|  5|
|1015552|Sentence|626411|             Cartoon|  6|
|1015610|Sentence|626411|             Cartoon| 64|
|1015827|Sentence|626413|Charlton Athletic...|190|
|1015968|Sentence|626414|          Battleship|135|
|1016299|Sentence|626417|          Bangladesh| 93|
|1016518|Sentence|626417|          Bangladesh|312|
|1016816|Sentence|626417|          Bangladesh|610|
|1017045|Sentence|626418|               Ghost| 40|
|1017074|Sentence|626418|               Ghost| 69|
|1017462|Sentence|626419|United States For...|133|
|1017551|Sentence|626421|         Amoxicillin| 38|
|1017716|Sentence|626422|         El Salvador|112|
|1017805|Sentence|626422|         El Salvador|201|
|1017849|Sentence|626422|      

In [18]:
### clause---attributes
clause_verticesText_test = verticesText1.filter("nodeType = 'Clause'")
clause_edges = edgesText3.filter("label = 'contains the clause'")


In [20]:
# combine dst_id
clause_verticesTextJ_test = clause_verticesText_test.join(clause_edges.select("src_id","src_nodeType","dst_id","dst_nodeType"), clause_verticesText_test.id==clause_edges.dst_id, "inner")
clause_verticesTextJ_test = clause_verticesTextJ_test.drop("src_nodeType").drop("dst_id").drop("dst_nodeType")


In [28]:
clause_verticesTextJ_test.show()

+-------+--------+--------------------+-----+--------------------+-------+
|     id|nodeType|               attr1|attr2|               attr3| src_id|
+-------+--------+--------------------+-----+--------------------+-------+
| 100010|  Clause|("Kerensky", "ord...|  SVO|            Kerensky|1234113|
|1000240|  Clause|("to the paternal...|  SVC|to the paternal h...|3875588|
|1000280|  Clause|("roughly speakin...|  SVO|roughly speaking ...|3650624|
|1000665|  Clause|("sympatric speci...|  SVC|sympatric speciation|2068926|
|1000795|  Clause|("The growth of t...|  SVO|The growth of the...| 467648|
|1000839|  Clause|("the priests", "...|  SVA|         the priests| 233944|
|1000888|  Clause|("the absolute va...|  SVC|  the absolute value|4022932|
| 100140|  Clause|("A memorial plaq...|  SVA|   A memorial plaque|1212133|
|1001866|  Clause|("1QDan", "is", "...|  SVC|               1QDan| 452611|
|1002011|  Clause|("the use of Afri...|  SVC|the use of Africa...|1487892|
|1002185|  Clause|("he", 

In [21]:
sen_verticesTextJ=sen_verticesText.withColumnRenamed("id","senid")##important, need to keep different column name with others when "JOIN"

In [22]:
# combine src_id
clause_verticesText_test1 = clause_verticesTextJ_test.join(sen_verticesTextJ.select("senid","did","dtitle","sid"), clause_verticesTextJ_test.src_id==sen_verticesTextJ.senid, "inner")
clause_verticesText_test2 = clause_verticesText_test1.drop("attr2").drop("attr3").drop("src_id").drop("sen_id")
clause_verticesText = clause_verticesText_test2

In [31]:
clause_verticesText.show()

+-------+--------+--------------------+-------+------+--------------------+---+
|     id|nodeType|               attr1|  senid|   did|              dtitle|sid|
+-------+--------+--------------------+-------+------+--------------------+---+
|2911087|  Clause|("Both the proces...|1014942|626404|              Genome|102|
|1732278|  Clause|("alcohols", "are...|1015263|626407|             Alcohol| 99|
|3236586|  Clause|("cartoon", "came...|1015551|626411|             Cartoon|  5|
|3493546|  Clause|("to animated fil...|1015552|626411|             Cartoon|  6|
|3493545|  Clause|("it", "began", "...|1015552|626411|             Cartoon|  6|
|3327830|  Clause|("Thomson", "capi...|1015610|626411|             Cartoon| 64|
|1230382|  Clause|("The ground-shar...|1015827|626413|Charlton Athletic...|190|
|2658479|  Clause|("the Austro-Hung...|1015968|626414|          Battleship|135|
|2658478|  Clause|("The Adriatic", ...|1015968|626414|          Battleship|135|
| 276441|  Clause|("British rule", ...|1

In [23]:
## change clause to predicate
clause_verticesText1 = clause_verticesText.withColumn("attr1", f.split(clause_verticesText['attr1'], ","))
predicate_verticesText = clause_verticesText.withColumn("predicate", f.split("attr1", ",")[1])
predicate_verticesText1 = predicate_verticesText.select("*", f.translate(f.col("predicate"), "[\"()]", "").alias("predicate1"))
predicate_verticesText2 = predicate_verticesText1.withColumn("nodeType1",f.lit("Predicate"))
predicate_verticesText = predicate_verticesText2.select("id","nodeType1","did","dtitle","sid","predicate1")

In [95]:
predicate_verticesText=predicate_verticesText.withColumnRenamed("nodeType1","nodeType").withColumnRenamed("predicate1","predicate")

In [102]:
predicate_verticesText.show()

+-------+---------+------+--------------------+---+-----------------+
|     id| nodeType|   did|              dtitle|sid|        predicate|
+-------+---------+------+--------------------+---+-----------------+
|2911087|Predicate|626404|              Genome|102|       can result|
|1732278|Predicate|626407|             Alcohol| 99|              are|
|3236586|Predicate|626411|             Cartoon|  5|             came|
|3493546|Predicate|626411|             Cartoon|  6|        resembled|
|3493545|Predicate|626411|             Cartoon|  6|            began|
|3327830|Predicate|626411|             Cartoon| 64|      capitalized|
|1230382|Predicate|626413|Charlton Athletic...|190|              was|
|2658479|Predicate|626414|          Battleship|135|         remained|
|2658478|Predicate|626414|          Battleship|135|              was|
| 276441|Predicate|626417|          Bangladesh| 93|        displaced|
| 276440|Predicate|626417|          Bangladesh| 93|        broke out|
| 464234|Predicate|6

In [24]:
### mention---mention edges
entity_edges = edgesText3.filter("label = 'is disambiguated as'")
NewMentionEdges = entity_edges.groupBy("dst_id").agg(f.collect_list('src_id').alias('NewMentionEdges'))
NewMentionEdges = NewMentionEdges.withColumnRenamed("NewMentionEdges","Mention")
NewMentionEdges = NewMentionEdges.select("Mention")
NewMentionEdges = NewMentionEdges.withColumn("Mention1",NewMentionEdges.Mention)
NewMentionEdgesJ = NewMentionEdges.withColumn("Mention1", f.explode("Mention1"))

NewMentionEdgesJ1 = NewMentionEdgesJ.withColumn("Mention", f.explode("Mention"))
NewMentionEdgesJ2 = NewMentionEdgesJ1.filter(NewMentionEdgesJ1.Mention!=NewMentionEdgesJ1.Mention1)


In [35]:
NewMentionEdgesJ2.show()

+-------+--------+
|Mention|Mention1|
+-------+--------+
|4417364| 4417352|
|4417356| 4417352|
|7915519| 4417352|
|7171576| 4417352|
|7171574| 4417352|
|4417352| 4417364|
|4417356| 4417364|
|7915519| 4417364|
|7171576| 4417364|
|7171574| 4417364|
|4417352| 4417356|
|4417364| 4417356|
|7915519| 4417356|
|7171576| 4417356|
|7171574| 4417356|
|4417352| 7915519|
|4417364| 7915519|
|4417356| 7915519|
|7171576| 7915519|
|7171574| 7915519|
+-------+--------+
only showing top 20 rows



In [36]:
##add type node

In [25]:
sen_verticesText_test=verticesText1.filter("nodeType = 'Sentence'")

In [26]:
sen_Type_verticesText_test=sen_verticesText_test.select("attr2")

In [39]:
sen_Type_verticesText_test.show()

+--------------------+
|               attr2|
+--------------------+
|Politics of Burun...|
|Executive power i...|
|Legislative power...|
|The political lan...|
|The current Presi...|
|Nkurunziza was th...|
|In November 1995,...|
|In July 1996, for...|
|He declared himse...|
|Widespread condem...|
|Buyoya agreed in ...|
|Nonetheless, figh...|
|In June 1998, Buy...|
|After facilitator...|
|Under Mandela the...|
|In April 2015 the...|
|Protests in the c...|
|The president is ...|
|He nominates two ...|
|The National Asse...|
+--------------------+
only showing top 20 rows



In [27]:
from pyspark.sql.functions import udf

In [28]:
from pyspark.sql.types import StringType

In [29]:
def replace_symbols(s):
	s=s.replace('(',' ')
	s=s.replace(')',' ')
	s=s.replace('[',' ')
	s=s.replace(']',' ')
	s=s.replace('{',' ')
	s=s.replace('}',' ')
	s=s.replace('|',' ')
	s=s.replace('"',' ')
	s=s.strip()
	return s

In [30]:
replaceS_udf=udf(replace_symbols,StringType())

In [31]:
sen_Type_verticesText_test1 = sen_Type_verticesText_test.select("attr2",replaceS_udf(sen_Type_verticesText_test["attr2"]).alias("RS"))

In [32]:
sen_Type_verticesText_test1 = sen_Type_verticesText_test.withColumn("RPS",replaceS_udf(sen_Type_verticesText_test["attr2"]))

In [46]:
# sen_Type_verticesText_test1 =sen_Type_verticesText_test.select(gtQSub.replace_symbols("attr2"))

In [83]:
sen_Type_verticesText_test1.show()

+--------------------+--------------------+
|               attr2|                 RPS|
+--------------------+--------------------+
|Politics of Burun...|Politics of Burun...|
|Executive power i...|Executive power i...|
|Legislative power...|Legislative power...|
|The political lan...|The political lan...|
|The current Presi...|The current Presi...|
|Nkurunziza was th...|Nkurunziza was th...|
|In November 1995,...|In November 1995,...|
|In July 1996, for...|In July 1996, for...|
|He declared himse...|He declared himse...|
|Widespread condem...|Widespread condem...|
|Buyoya agreed in ...|Buyoya agreed in ...|
|Nonetheless, figh...|Nonetheless, figh...|
|In June 1998, Buy...|In June 1998, Buy...|
|After facilitator...|After facilitator...|
|Under Mandela the...|Under Mandela the...|
|In April 2015 the...|In April 2015 the...|
|Protests in the c...|Protests in the c...|
|The president is ...|The president is ...|
|He nominates two ...|He nominates two ...|
|The National Asse...|The Nation

In [33]:
sen_Type_verticesText_test1=sen_Type_verticesText_test1.drop("attr2")

In [135]:
# sc.addPyFile('/Users/jeanxu/Documents/UniLU/0_MasterThesis/5_Notes/QA_Notes/1_Test/python/project/library/hearstPatterns.py')

In [162]:
# import hearstPatterns as hP

In [163]:
# h = hP.HearstPatterns(extended=True)

In [48]:
#test find_hyponyms
# h.find_hyponyms("Forty-four percent of patients with uveitis had one or more identifiable signs or symptoms, such as red eye, ocular pain, visual acuity, or photophobia, in order of decreasing frequency.")

In [154]:
# from hearstPatterns import HearstPatterns

In [155]:
# h=HearstPatterns(extended=True)

In [156]:
#test find_hyponyms
# x=h.find_hyponyms("Forty-four percent of patients with uveitis had one or more identifiable signs or symptoms, such as red eye, ocular pain, visual acuity, or photophobia, in order of decreasing frequency.")

In [120]:
#test use pandas
# sen_Type_verticesText_test2["RPS"][0]

In [121]:
# sen_Type_verticesText_test2["RPS"].count()

In [122]:
# h.find_hyponyms(sen_Type_verticesText_test2["RPS"][0])

In [165]:
#cost a lot of time give up
# sen_Type_verticesText_test2=sen_Type_verticesText_test1.toPandas()
# HP = set()
# for i in range(sen_Type_verticesText_test2["RPS"].count()):
#     HP1 = set(h.find_hyponyms(sen_Type_verticesText_test2["RPS"][i]))#
#     HP = HP.union(HP1)

In [35]:
import re
import string
import nltk
from nltk.tag.perceptron import PerceptronTagger

class HearstPatterns(object):

    def __init__(self, extended = False):
        self.__chunk_patterns = r""" #  helps us find noun phrase chunks
                NP: {<DT|PP\$>?<JJ>*<NN>+}
                    {<NNP>+}
                    {<NNS>+}
        """

        self.__np_chunker = nltk.RegexpParser(self.__chunk_patterns) # create a chunk parser 

        # now define the Hearst patterns
        # format is <hearst-pattern>, <general-term>
        # so, what this means is that if you apply the first pattern, the firsr Noun Phrase (NP)
        # is the general one, and the rest are specific NPs
        self.__hearst_patterns = [
                ("(NP_\w+ (, )?such as (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(such NP_\w+ (, )?as (NP_\w+ ?(, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?other NP_\w+)", "last"),
                ("(NP_\w+ (, )?including (NP_\w+ ?(, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?especially (NP_\w+ ?(, )?(and |or )?)+)", "first"),
            ]

        if extended:
            self.__hearst_patterns.extend([
                ("((NP_\w+ ?(, )?)+(and |or )?any other NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?some other NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?is a NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?was a NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?were a NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?are a NP_\w+)", "last"),
                ("(NP_\w+ (, )?like (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("such (NP_\w+ (, )?as (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?like other NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?one of the NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?one of these NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?one of those NP_\w+)", "last"),
                ("examples of (NP_\w+ (, )?is (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("examples of (NP_\w+ (, )?are (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?are examples of NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?is example of NP_\w+)", "last"),
                ("(NP_\w+ (, )?for example (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?wich is called NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?which is named NP_\w+)", "last"),
                ("(NP_\w+ (, )?mainly (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?mostly (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?notably (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?particularly (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?principally (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?in particular (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?except (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?other than (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?e.g. (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?i.e. (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?a kind of NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?kinds of NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?form of NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?forms of NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?which looks like NP_\w+)", "last"),
                ("((NP_\w+ ?(, )?)+(and |or )?which sounds like NP_\w+)", "last"),
                ("(NP_\w+ (, )?which are similar to (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?which is similar to (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?examples of this is (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?examples of this are (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?types (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )? NP_\w+ types)", "last"),
                ("(NP_\w+ (, )?whether (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(compare (NP_\w+ ?(, )?)+(and |or )?with NP_\w+)", "last"),
                ("(NP_\w+ (, )?compared to (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("(NP_\w+ (, )?among them (NP_\w+ ? (, )?(and |or )?)+)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?as NP_\w+)", "last"),
                ("(NP_\w+ (, )? (NP_\w+ ? (, )?(and |or )?)+ for instance)", "first"),
                ("((NP_\w+ ?(, )?)+(and |or )?sort of NP_\w+)", "last"),
            ])

        self.__pos_tagger = PerceptronTagger()
        
    def prepare(self, rawtext):
        sentences = nltk.sent_tokenize(rawtext.strip()) # NLTK default sentence segmenter
        sentences = [nltk.word_tokenize(sent) for sent in sentences] # NLTK word tokenizer
        sentences = [self.__pos_tagger.tag(sent) for sent in sentences] # NLTK POS tagger

        return sentences

    def chunk(self, rawtext):
        sentences = self.prepare(rawtext.strip())

        all_chunks = []
        for sentence in sentences:
            chunks = self.__np_chunker.parse(sentence) # parse the example sentence
            #for chunk in chunks:
            #   print(str(chunk))
            all_chunks.append(self.prepare_chunks(chunks))
        return all_chunks

    def prepare_chunks(self, chunks):
        # basically, if the chunk is NP, keep it as a string taht starts w/ NP and replace " " with _
        # otherwise, keep the word.
        # remove punct
        # this is all done to make it super easy to apply the Hearst patterns...

        terms = []
        for chunk in chunks:
            label = None
            try: # gross hack to see if the chunk is simply a word or a NP, as we want. But non-NP fail on this method call
                label = chunk.label()
            except:
                pass

            if label is None: #means one word...
                token = chunk[0]
                pos = chunk[1]
                if pos in ['.', ':', '-', '_']:
                    continue
                terms.append(token)
            else:
                np = "NP_"+"_".join([a[0] for a in chunk]) #This makes it easy to apply the Hearst patterns later
                terms.append(np)
        return ' '.join(terms)

    def replace_np_sequences(self, sentence):
        words = ""
        first_word_in_sequence = False
        for word in nltk.word_tokenize(sentence.replace("NP_", "_")):
            if word[0] == "_":
                if not first_word_in_sequence:
                    word = "NP" + word
                    first_word_in_sequence = True
                    words = words + " " + word
                else:
                    words += word
            else:
                words = words + " " + word
                first_word_in_sequence = False
        return words.strip()

    """
        This is the main entry point for this code.
        It takes as input the rawtext to process and returns a list of tuples (specific-term, general-term)
        where each tuple represents a hypernym pair.

    """
    def find_hyponyms(self, rawtext):

        hyponyms = []
        np_tagged_sentences = self.chunk(rawtext)
        #print "NP tagged-->",np_tagged_sentences

        for raw_sentence in np_tagged_sentences:
            # two or more NPs next to each other should be merged into a single NP, it's a chunk error

            # find any N consecutive NP_ and merge them into one...
            # So, something like: "NP_foo NP_bar blah blah" becomes "NP_foo_bar blah blah"
            sentence = self.replace_np_sequences(raw_sentence)

            for (hearst_pattern, parser) in self.__hearst_patterns:
                matches = re.search(hearst_pattern, sentence)
                if matches:
                    match_str = matches.group(0)

                    nps = [a for a in match_str.split() if a.startswith("NP_")]

                    if parser == "first":
                        general = nps[0]
                        specifics = nps[1:]
                    else:
                        general = nps[-1]
                        specifics = nps[:-1]
                        #print(str(general))
                        #print(str(nps))

                    for i in range(len(specifics)):
                        #print("%s, %s" % (specifics[i], general))
                        hs = self.clean_hyponym_term(specifics[i])+"-"+self.clean_hyponym_term(general)
                        hyponyms.append(hs)
                        # hyponyms.append((self.clean_hyponym_term(general),self.clean_hyponym_term(general)))
                        # return hyponyms
        if hyponyms!=[]:
            return hyponyms

    # from pyspark.sql.functions import udf
    # from pyspark.sql.types import ArrayType, StringType

    # find_hyponyms_udf = udf(lambda x: find_hyponyms(x), ArrayType(StringType()))

    def clean_hyponym_term(self, term):
        # good point to do the stemming or lemmatization
        return term.replace("NP_","").replace("_", " ")


In [36]:
H=HearstPatterns(extended=True)

In [37]:
H.find_hyponyms("Forty-four percent of patients with uveitis had one or more identifiable signs or symptoms, such as red eye, ocular pain, visual acuity, or photophobia, in order of decreasing frequency.")

['red eye-symptoms',
 'ocular pain-symptoms',
 'visual acuity-symptoms',
 'photophobia-symptoms']

In [38]:
H.find_hyponyms("apple")

In [39]:
from pyspark.sql.types import ArrayType

In [40]:
find_hyponyms_udf = udf(H.find_hyponyms,ArrayType(StringType()))

In [41]:
sen_Type_verticesText_test2 = sen_Type_verticesText_test1.withColumn("Tp",find_hyponyms_udf(sen_Type_verticesText_test1["RPS"]))

In [42]:
sen_Type_verticesText_test2.show()

+--------------------+--------------------+
|                 RPS|                  Tp|
+--------------------+--------------------+
|Politics of Burun...|                null|
|Executive power i...|                null|
|Legislative power...|                null|
|The political lan...|                null|
|The current Presi...|                null|
|Nkurunziza was th...|                null|
|In November 1995,...|                null|
|In July 1996, for...|                null|
|He declared himse...|                null|
|Widespread condem...|                null|
|Buyoya agreed in ...|                null|
|Nonetheless, figh...|                null|
|In June 1998, Buy...|                null|
|After facilitator...|[Nelson Mandela-F...|
|Under Mandela the...|                null|
|In April 2015 the...|                null|
|Protests in the c...|                null|
|The president is ...|                null|
|He nominates two ...|                null|
|The National Asse...|          

In [43]:
HType=sen_Type_verticesText_test2.select("Tp")

In [47]:
HType1=HType.dropna()

In [48]:
HType1.show()

+--------------------+
|                  Tp|
+--------------------+
|[Nelson Mandela-F...|
|[ADTs-the abstrac...|
|   [data-data types]|
|[relations-constr...|
|[lattices-structu...|
|[S-an explicit in...|
|    [ADT-parameters]|
|[the structure-a ...|
|[the old state-an...|
|[the same input s...|
|[ADT instance use...|
|[ADT operations-m...|
|  [interfaces-types]|
|[C-an imperative ...|
|[Awk-many scripti...|
|[Alpha engineerin...|
|[Vinod Dham-engin...|
|  [Athlons-a result]|
|[core voltages-Th...|
|[applications-an ...|
+--------------------+
only showing top 20 rows



In [51]:
HType2=HType1.withColumn("Tp1", f.explode("Tp"))

In [52]:
HType2.show()

+--------------------+--------------------+
|                  Tp|                 Tp1|
+--------------------+--------------------+
|[Nelson Mandela-F...|Nelson Mandela-Fa...|
|[ADTs-the abstrac...|ADTs-the abstraction|
|   [data-data types]|     data-data types|
|[relations-constr...|relations-constra...|
|[lattices-structu...| lattices-structures|
|[lattices-structu...|   groups-structures|
|[S-an explicit in...|S-an explicit ins...|
|    [ADT-parameters]|      ADT-parameters|
|[the structure-a ...|the structure-a s...|
|[the old state-an...|the old state-an ...|
|[the same input s...|the same input st...|
|[ADT instance use...|ADT instance uses...|
|[ADT operations-m...|ADT operations-me...|
|  [interfaces-types]|    interfaces-types|
|[C-an imperative ...|C-an imperative l...|
|[Awk-many scripti...|Awk-many scriptin...|
|[Awk-many scripti...|Lua-many scriptin...|
|[Awk-many scripti...|Perl-many scripti...|
|[Alpha engineerin...|Alpha engineering...|
|[Vinod Dham-engin...|Vinod Dham

In [54]:
df_temp = HType2.withColumn("MentionName", f.split("Tp1", "-")[0])


In [55]:
HType3 = df_temp.withColumn("Type", f.split("Tp1", "-")[1])

In [59]:
HType3=HType3.drop("Tp").drop("Tp1")

In [60]:
HType3.show()

+--------------------+--------------------+
|         MentionName|                Type|
+--------------------+--------------------+
|      Nelson Mandela|         Facilitator|
|                ADTs|     the abstraction|
|                data|          data types|
|           relations|         constraints|
|            lattices|          structures|
|              groups|          structures|
|                   S|an explicit insta...|
|                 ADT|          parameters|
|       the structure|   a separate entity|
|       the old state|         an argument|
|the same input st...|           arguments|
|   ADT instance uses|          a function|
|      ADT operations|             methods|
|          interfaces|               types|
|                   C|an imperative lan...|
|                 Awk|many scripting la...|
|                 Lua|many scripting la...|
|                Perl|many scripting la...|
|Alpha engineering...|              Compaq|
|          Vinod Dham|          

In [75]:
# HType=HType.distinct()

In [61]:
men_verticesText_1=verticesText1.filter("nodeType = 'Mention'")

In [64]:
men_verticesText_1=men_verticesText_1.drop("attr3")

In [65]:
men_verticesText_1.show()

+-----+--------+--------------------+------------+
|   id|nodeType|               attr1|       attr2|
+-----+--------+--------------------+------------+
|24975| Mention|       Freedom House|        MISC|
|24976| Mention|                 NGO|ORGANIZATION|
|24977| Mention|              Obiang|      PERSON|
|24978| Mention|       Freedom House|        MISC|
|24979| Mention|                  US|    LOCATION|
|24980| Mention|       Freedom House|        MISC|
|24981| Mention|                Root|      PERSON|
|24982| Mention|           Queen Ask|        MISC|
|24983| Mention|     Act of Congress|        MISC|
|24984| Mention|          Public Law|        MISC|
|24985| Mention|              Darwin|      PERSON|
|24986| Mention|Evidence as to Ma...|        MISC|
|24987| Mention|       Thomas Huxley|      PERSON|
|24988| Mention|             Spanish|    LOCATION|
|24989| Mention|       Mariano Rajoy|      PERSON|
|24990| Mention|            European|    LOCATION|
|24991| Mention|        Dragon 

In [66]:
mention_edges_1=edgesText3.filter("label = 'contains the mention'")

In [69]:
mention_edges_1=mention_edges_1.drop("src").drop("dst").drop("label")

In [70]:
mention_edges_1.show()

+-------+------------+-------+------------+
| src_id|src_nodeType| dst_id|dst_nodeType|
+-------+------------+-------+------------+
|2073024|      Clause|1007462|     Mention|
|2849688|      Clause|1007636|     Mention|
|2568628|      Clause|1007678|     Mention|
| 117777|      Clause|1008797|     Mention|
|1759866|      Clause|1009080|     Mention|
|2846388|      Clause|1009129|     Mention|
| 564274|      Clause|1009177|     Mention|
| 935474|      Clause|1009407|     Mention|
|1878077|      Clause|1009511|     Mention|
|3643422|      Clause|1009562|     Mention|
|1860471|      Clause|1010103|     Mention|
|1860472|      Clause|1010108|     Mention|
|2076128|      Clause|1010262|     Mention|
| 588258|      Clause|1010351|     Mention|
|2658225|      Clause|1010503|     Mention|
| 321989|      Clause|1010766|     Mention|
| 713827|      Clause|1010896|     Mention|
|3699391|      Clause|1011171|     Mention|
|3699391|      Clause|1011172|     Mention|
|1240648|      Clause|1011187|  

In [76]:
# combine dst_id
mention_verticesText2 = mention_edges_1.join(men_verticesText_1, mention_edges_1.dst_id==men_verticesText_1.id, "inner")
# clause_verticesTextJ_test = clause_verticesTextJ_test.drop("src_nodeType").drop("dst_id").drop("dst_nodeType")


In [79]:
mention_verticesText2=mention_verticesText2.drop("id").drop("nodeType")

In [80]:
mention_verticesText2.show()

+-------+------------+-------+------------+--------------------+--------+
| src_id|src_nodeType| dst_id|dst_nodeType|               attr1|   attr2|
+-------+------------+-------+------------+--------------------+--------+
|2073024|      Clause|1007462|     Mention|     Newington Youth|LOCATION|
|2849688|      Clause|1007636|     Mention|American Film Ins...|    MISC|
|2568628|      Clause|1007678|     Mention|               Italy|LOCATION|
| 117777|      Clause|1008797|     Mention|              Church|    MISC|
|1759866|      Clause|1009080|     Mention|             Laurier|  PERSON|
|2846388|      Clause|1009129|     Mention|Anders Behring Br...|  PERSON|
| 564274|      Clause|1009177|     Mention|     White Stockings|    MISC|
| 935474|      Clause|1009407|     Mention|             Eleanor|  PERSON|
|1878077|      Clause|1009511|     Mention|  Bristol University|LOCATION|
|3643422|      Clause|1009562|     Mention|               Earth|LOCATION|
|1860471|      Clause|1010103|     Men

In [81]:
# combine src_id
mention_verticesText3 = mention_verticesText2.join(predicate_verticesText, mention_verticesText2.src_id==predicate_verticesText.id, "inner")

In [85]:
mention_verticesText3=mention_verticesText3.drop("src_id").drop("src_nodeType").drop("id").drop("nodeType")

In [88]:
mention_verticesText3=mention_verticesText3.drop("predicate")

In [90]:
mention_verticesText3=mention_verticesText3.withColumnRenamed("dst_id","id").withColumnRenamed("dst_nodeType","nodeType")

In [91]:
mention_verticesText3.show()

+-------+--------+--------------------+--------+-------+-------------------+---+
|     id|nodeType|               attr1|   attr2|    did|             dtitle|sid|
+-------+--------+--------------------+--------+-------+-------------------+---+
|2216143| Mention|            Kerensky|  PERSON| 606008| Alexander Kerensky| 34|
|2216144| Mention| February Revolution|    MISC| 606008| Alexander Kerensky| 34|
|3579976| Mention|           Cathedral|    MISC| 561719|  Claude Auchinleck|101|
|3579975| Mention|             St Paul|LOCATION| 561719|  Claude Auchinleck|101|
|2086736| Mention|             African|LOCATION|2444643|       Black people|183|
|2086735| Mention|                  US|LOCATION|2444643|       Black people|183|
|3571984| Mention|Nordic Chess Cham...|    MISC|2420916|   Aron Nimzowitsch| 13|
|3571985| Mention|          Copenhagen|LOCATION|2420916|   Aron Nimzowitsch| 13|
|1679175| Mention|      Ottoman Sultan|  PERSON| 851331|Christopher Báthory| 27|
|1679176| Mention|        Tr

In [97]:
HType4=HType3.distinct()

In [93]:
##join type by heart
mention_verticesText4=mention_verticesText3.join(HType3, mention_verticesText3.attr1==HType3.MentionName,"left")

In [100]:
mention_verticesText5=mention_verticesText3.join(HType4, mention_verticesText3.attr1==HType4.MentionName,"left")

In [94]:
mention_verticesText4.show()

+-------+--------+--------------------+--------+-------+-----------------+---+-----------+-------------------+
|     id|nodeType|               attr1|   attr2|    did|           dtitle|sid|MentionName|               Type|
+-------+--------+--------------------+--------+-------+-----------------+---+-----------+-------------------+
|3559179| Mention|            Artesian|LOCATION|1439190|     Ellis Island| 39|       null|               null|
|2021479| Mention|              Baloch|  PERSON|1511954|      Afghanistan|388|       null|               null|
|1884303| Mention|      Blackford Hill|  PERSON|2081882|        Edinburgh| 98|       null|               null|
|2242617| Mention|      Caesar Cardini|  PERSON|1235788|     Caesar salad| 16|       null|               null|
|2809399| Mention|Combatant Status ...|    MISC|3625401|     Abu Zubaydah|176|       null|               null|
|4039815| Mention|            Disraeli|  PERSON|2756547|Benjamin Disraeli|159|   Disraeli|           Disraeli|
|

In [101]:
mention_verticesText5.show()

+-------+--------+--------------------+--------+-------+-----------------+---+-----------+--------------------+
|     id|nodeType|               attr1|   attr2|    did|           dtitle|sid|MentionName|                Type|
+-------+--------+--------------------+--------+-------+-----------------+---+-----------+--------------------+
|3559179| Mention|            Artesian|LOCATION|1439190|     Ellis Island| 39|       null|                null|
|2021479| Mention|              Baloch|  PERSON|1511954|      Afghanistan|388|       null|                null|
|1884303| Mention|      Blackford Hill|  PERSON|2081882|        Edinburgh| 98|       null|                null|
|2242617| Mention|      Caesar Cardini|  PERSON|1235788|     Caesar salad| 16|       null|                null|
|2809399| Mention|Combatant Status ...|    MISC|3625401|     Abu Zubaydah|176|       null|                null|
|4039815| Mention|            Disraeli|  PERSON|2756547|Benjamin Disraeli|159|   Disraeli| Conservative 

In [ ]:
###alignment 'Predicate' / 'Type' / 'mention' alignment edges

In [ ]:
#Predicate alignment
#predicate=auxlite(is, was...) & predicate !=auxlist

#1-get all predicat with id in column
#2-delete all auxlist(was, is, are)
#3-find same predicate in column (with different id)
#4-get predicate glove
#5-get weight and generate edge

In [111]:
predicate_alignment1=predicate_verticesText.select("id","predicate")

In [104]:
predicate_alignment1.show()

+-------+-----------------+
|     id|        predicate|
+-------+-----------------+
|2911087|       can result|
|1732278|              are|
|3236586|             came|
|3493546|        resembled|
|3493545|            began|
|3327830|      capitalized|
|1230382|              was|
|2658479|         remained|
|2658478|              was|
| 276441|        displaced|
| 276440|        broke out|
| 464234|      was enacted|
| 427430|        were used|
| 386149|             died|
| 386148|     never leaves|
| 386147|          believe|
|3457439|          is said|
|1880973| would be amended|
|1880971|         prompted|
|1880972|  had overreached|
+-------+-----------------+
only showing top 20 rows



In [ ]:
#2-delete all auxlist(was, is, are)

In [137]:
from pyspark.ml.feature import StopWordsRemover,RegexTokenizer, Word2Vec

In [126]:
predicate_alignment1_regexTokenizer = RegexTokenizer(inputCol='predicate', outputCol='predicate1', pattern='\\W')

In [156]:
aux_stop_list = ['be', 'am', 'being', 'been', 'is', 'are', 'was', 'were', 'has', 'have', 'had', 'having', 'do', 'does', 'did',
     'done', 'will', 'would', 'shall', 'should', 'can', 'could', 'dare', 'may', 'might', 'must', 'need', 'ought', 
     'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and', 'any', 'are', 'aren\'t', 'as', 'at',
     'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can\'t', 'cannot', 'could',
     'couldn\'t', 'did', 'didn\'t', 'do', 'does', 'doesn\'t', 'doing', 'don\'t', 'down', 'during', 'each', 'few', 'for',
     'from', 'further', 'hadn\'t', 'hasn\'t', 'haven\'t', 'he', 'he\'d', 'he\'ll',
     'he\'s', 'her', 'here', 'here\'s', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'how\'s', 'i', 'i\'d',
     'i\'ll', 'i\'m', 'i\'ve', 'if', 'in', 'into', 'is', 'isn\'t', 'it', 'it\'s', 'its', 'itself', 'let\'s',
     'more', 'most', 'mustn\'t','myself', 'no', 'nor', 'not', 'of', 'off', 'on', 'once', 'only', 'or', 'other',
     'ought', 'ours','ourselves', 'out', 'over', 'own', 'same', 'shan\'t', 'she\'d', 'she\'ll', 'she\'s',
     'should', 'shouldn\'t', 'so', 'some', 'such', 'than', 'that', 'that\'s', 'the', 'theirs',
     'themselves', 'then', 'there\'s', 'they\'d', 'they\'ll', 'they\'re', 'they\'ve', 'this',
     'those', 'through', 'to', 'too', 'under', 'until', 'up', 'very', 'wasn\'t', 'we\'d', 'we\'ll',
     'we\'re', 'we\'ve', 'were', 'weren\'t', 'what', 'what\'s', 'when', 'when\'s', 'where', 'where\'s', 'which',
     'while', 'who', 'who\'s', 'whom', 'why', 'why\'s', 'with', 'won\'t', 'wouldn\'t', 'you\'d',
     'you\'ll', 'you\'re', 'you\'ve', 'your', 'yours', 'yourself', 'yourselves']

In [157]:
predicate_alignment1_remove = StopWordsRemover(inputCol='predicate1', outputCol='predicate2').setStopWords(aux_stop_list)

In [121]:
from pyspark.ml import Pipeline

In [158]:
pipeline = Pipeline(stages=[predicate_alignment1_regexTokenizer,predicate_alignment1_remove])

In [159]:
pipeline_fit = pipeline.fit(predicate_alignment1)

In [160]:
predicate_alignment2 = pipeline_fit.transform(predicate_alignment1)

In [161]:
predicate_alignment2.show()

+-------+-----------------+--------------------+---------------+
|     id|        predicate|          predicate1|     predicate2|
+-------+-----------------+--------------------+---------------+
|2911087|       can result|       [can, result]|       [result]|
|1732278|              are|               [are]|             []|
|3236586|             came|              [came]|         [came]|
|3493546|        resembled|         [resembled]|    [resembled]|
|3493545|            began|             [began]|        [began]|
|3327830|      capitalized|       [capitalized]|  [capitalized]|
|1230382|              was|               [was]|             []|
|2658479|         remained|          [remained]|     [remained]|
|2658478|              was|               [was]|             []|
| 276441|        displaced|         [displaced]|    [displaced]|
| 276440|        broke out|        [broke, out]|        [broke]|
| 464234|      was enacted|      [was, enacted]|      [enacted]|
| 427430|        were use

In [163]:
predicate_alignment3 = predicate_alignment2.select("id","predicate2")

In [164]:
predicate_alignment3.show()

+-------+---------------+
|     id|     predicate2|
+-------+---------------+
|2911087|       [result]|
|1732278|             []|
|3236586|         [came]|
|3493546|    [resembled]|
|3493545|        [began]|
|3327830|  [capitalized]|
|1230382|             []|
|2658479|     [remained]|
|2658478|             []|
| 276441|    [displaced]|
| 276440|        [broke]|
| 464234|      [enacted]|
| 427430|         [used]|
| 386149|         [died]|
| 386148|[never, leaves]|
| 386147|      [believe]|
|3457439|         [said]|
|1880973|      [amended]|
|1880971|     [prompted]|
|1880972|  [overreached]|
+-------+---------------+
only showing top 20 rows



In [165]:
predicate_alignment3 = predicate_alignment3.withColumn("size", f.size(f.col("predicate2")))

In [153]:
predicate_alignment3.show()

+-------+---------------+----+
|     id|     predicate2|size|
+-------+---------------+----+
|2911087|       [result]|   1|
|1732278|             []|   0|
|3236586|         [came]|   1|
|3493546|    [resembled]|   1|
|3493545|        [began]|   1|
|3327830|  [capitalized]|   1|
|1230382|             []|   0|
|2658479|     [remained]|   1|
|2658478|             []|   0|
| 276441|    [displaced]|   1|
| 276440|   [broke, out]|   2|
| 464234|      [enacted]|   1|
| 427430|         [used]|   1|
| 386149|         [died]|   1|
| 386148|[never, leaves]|   2|
| 386147|      [believe]|   1|
|3457439|         [said]|   1|
|1880973|      [amended]|   1|
|1880971|     [prompted]|   1|
|1880972|  [overreached]|   1|
+-------+---------------+----+
only showing top 20 rows



In [166]:
predicate_alignment3 = predicate_alignment3.filter(f.col("size") != 0)

In [155]:
predicate_alignment3.show()

+-------+---------------+----+
|     id|     predicate2|size|
+-------+---------------+----+
|2911087|       [result]|   1|
|3236586|         [came]|   1|
|3493546|    [resembled]|   1|
|3493545|        [began]|   1|
|3327830|  [capitalized]|   1|
|2658479|     [remained]|   1|
| 276441|    [displaced]|   1|
| 276440|   [broke, out]|   2|
| 464234|      [enacted]|   1|
| 427430|         [used]|   1|
| 386149|         [died]|   1|
| 386148|[never, leaves]|   2|
| 386147|      [believe]|   1|
|3457439|         [said]|   1|
|1880973|      [amended]|   1|
|1880971|     [prompted]|   1|
|1880972|  [overreached]|   1|
|1443408|     [combined]|   1|
|1747188|      [started]|   1|
|3380427|        [drain]|   1|
+-------+---------------+----+
only showing top 20 rows



In [162]:
#1
predicate_alignment3.count()

2128482

In [167]:
#2
predicate_alignment3.count()

2070642

In [170]:
predicate_alignment3=predicate_alignment3.drop("size")

In [ ]:
#3-find same predicate in column (with different id)

In [171]:
predicate_alignment4=predicate_alignment3.withColumnRenamed("id","id2").withColumnRenamed("predicate2","predicate3")

In [172]:
predicate_alignment4=predicate_alignment4.join(predicate_alignment3,predicate_alignment3.predicate2==predicate_alignment4.predicate3,"inner")

In [173]:
predicate_alignment4.show()

+-------+--------------------+-------+--------------------+
|    id2|          predicate3|     id|          predicate2|
+-------+--------------------+-------+--------------------+
|2571288|            [0, 258]|2571288|            [0, 258]|
|1971975|         [0, 4, mag]|1971975|         [0, 4, mag]|
|5629309|[000, denominatio...|5629309|[000, denominatio...|
|5609685|       [000, greeks]|5609685|       [000, greeks]|
|5609685|       [000, greeks]|2691949|       [000, greeks]|
|5609685|       [000, greeks]|1645067|       [000, greeks]|
|2691949|       [000, greeks]|5609685|       [000, greeks]|
|2691949|       [000, greeks]|2691949|       [000, greeks]|
|2691949|       [000, greeks]|1645067|       [000, greeks]|
|1645067|       [000, greeks]|5609685|       [000, greeks]|
|1645067|       [000, greeks]|2691949|       [000, greeks]|
|1645067|       [000, greeks]|1645067|       [000, greeks]|
|2571009|[000, inhabitants...|2571009|[000, inhabitants...|
|2571009|[000, inhabitants...|2571011|[0

In [174]:
predicate_alignment4=predicate_alignment4.filter(predicate_alignment4.id2!=predicate_alignment4.id)

In [175]:
predicate_alignment4.show()#bidirection predict alignment weight=1

+-------+--------------------+-------+--------------------+
|    id2|          predicate3|     id|          predicate2|
+-------+--------------------+-------+--------------------+
|5609685|       [000, greeks]|2691949|       [000, greeks]|
|5609685|       [000, greeks]|1645067|       [000, greeks]|
|2691949|       [000, greeks]|5609685|       [000, greeks]|
|2691949|       [000, greeks]|1645067|       [000, greeks]|
|1645067|       [000, greeks]|5609685|       [000, greeks]|
|1645067|       [000, greeks]|2691949|       [000, greeks]|
|2571009|[000, inhabitants...|2571011|[000, inhabitants...|
|2571011|[000, inhabitants...|2571009|[000, inhabitants...|
|5096949|[000, members, ea...|5096948|[000, members, ea...|
|5096948|[000, members, ea...|5096949|[000, members, ea...|
|3564033|               [467]|3564032|               [467]|
|3564033|               [467]|6857259|               [467]|
|3564033|               [467]|6421152|               [467]|
|3564033|               [467]|5334293|  

In [139]:
predicate_alignment2_word2Vec = Word2Vec(vectorSize=300, minCount=0, inputCol="predicate2", outputCol="predicate3")

In [ ]:
predicate_alignment2_model = predicate_alignment2_word2Vec.fit(predicate_alignment2)

In [ ]:
predicate_alignment3=result = predicate_alignment2_model.transform(predicate_alignment2)

In [132]:
#predicate_alignment2=predicate_alignment1.select("id",f.translate(f.col("predicate"), aux_list, "").alias("predicate1"))

In [ ]:
gdict={}
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  
word_vectors = model.wv
gdict=word_vectors

In [133]:
# predicate_alignment2 = predicate_alignment1.filter(predicate_alignment1.predicate!=aux_list)

In [ ]:
def read_glove(world, gdict):
    veclen = 300
    g_pred = {}
    g_ent = {}
    g_type = {}
    g_ques = {}
    done = set()

    for n in G.nodes():
        if n not in done:
            done.add(n)
            nn = n.split(':')
            nw1 = nn[0].replace('-', ' ').split()
            avec = np.zeros(veclen)
            c = 0.0
            for el in nw1:
                if el in gdict and el.lower() not in stop_list:
                    # if option=='GLOVE':
                    #	avec=np.add(avec,gdict[el])
                    # else:
                    avec = np.add(avec, np.array(gdict[el]))
                    c += 1.0
            if c > 0:
                avec = np.divide(avec, c)

                if nn[1] == 'Predicate':
                    g_pred[n] = avec.tolist()
                else:
                    if nn[1] == 'Entity':
                        g_ent[n] = avec.tolist()
                    else:
                        g_type[n] = avec.tolist()
        # if verbose:
        # print len(G.nodes()),len(done)

    for n in q_ent:
        if n not in done:
            done.add(n)
            nw1 = n.replace('-', ' ').split()
            avec = np.zeros(veclen)
            c = 0.0
            for el in nw1:
                if el in gdict and el.lower() not in stop_list:
                    # if option=='GLOVE':
                    #	avec=np.add(avec,gdict[el])
                    # else:
                    avec = np.add(avec, np.array(gdict[el]))
                    c += 1.0
            if c > 0:
                avec = np.divide(avec, c)
                g_ques[n] = avec.tolist()

    return g_w